In [1]:
pip install pandas numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
#Change the path to local directory where the file will be saved that needs to be read.
path = 'C:/Users/91880/Desktop/karan/Dissertation'

In [3]:
#Import necessary libraries and sklearn modules to run the process.
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

In [4]:
# Read csv file.
ehw = pd.read_csv(path + '/ehw.csv', encoding= 'unicode_escape')

In [5]:
#This is how file looks like.
ehw.head()

,LCLid,time,energy_sum,stdorToU,Acorn,Acorn_grouped,temperatureMax,windBearing,icon,dewPoint,cloudCover,windSpeed,pressure,visibility,humidity,uvIndex,temperatureMin,moonPhase
0,MAC002542,17-Jan-13,5.543,Std,ACORN-E,Affluent,2.88,111,partly-cloudy-day,-2.88,0.31,1.01,1017.44,9.64,0.83,1,-3.86,0.2
1,MAC002543,17-Jan-13,5.270,Std,ACORN-E,Affluent,2.88,111,partly-cloudy-day,-2.88,0.31,1.01,1017.44,9.64,0.83,1,-3.86,0.2
2,MAC002545,17-Jan-13,2.015,Std,ACORN-E,Affluent,2.88,111,partly-cloudy-day,-2.88,0.31,1.01,1017.44,9.64,0.83,1,-3.86,0.2
3,MAC002546,17-Jan-13,7.329,Std,ACORN-E,Affluent,2.88,111,partly-cloudy-day,-2.88,0.31,1.01,1017.44,9.64,0.83,1,-3.86,0.2
4,MAC002548,17-Jan-13,4.479,Std,ACORN-E,Affluent,2.88,111,partly-cloudy-day,-2.88,0.31,1.01,1017.44,9.64,0.83,1,-3.86,0.2


In [6]:
#Picking up random 80,000 rows for faster processing as the dataset is quite large.
ehw1 = ehw.sample(n = 80000)

In [7]:
#The summary of whole dataset's statistical properties. 
#This helps to verify the count and look for any missing values if present, mean,max etc.
ehw1.describe()

,energy_sum,temperatureMax,windBearing,dewPoint,cloudCover,windSpeed,pressure,visibility,humidity,uvIndex,temperatureMin,moonPhase
count,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000,80000.000000
mean,9.669740,14.660954,194.789475,7.291363,0.480884,3.473870,1014.170897,11.282976,0.776547,2.861725,8.254284,0.505726
std,9.486219,6.638723,92.208320,5.081918,0.192934,1.632386,9.874580,2.386989,0.099358,1.912041,5.151034,0.287990
min,0.000000,-0.060000,0.000000,-7.840000,0.000000,0.400000,979.520000,1.480000,0.430000,0.000000,-5.640000,0.000000
25%,4.157000,9.780000,113.000000,3.710000,0.350000,2.300000,1008.500000,10.380000,0.710000,1.000000,4.090000,0.260000
50%,7.009000,14.560000,218.000000,7.790000,0.470000,3.360000,1014.560000,12.040000,0.790000,2.000000,8.480000,0.510000
75%,11.744000,19.640000,257.000000,11.190000,0.610000,4.420000,1020.900000,12.840000,0.860000,5.000000,12.480000,0.760000
max,262.115000,32.400000,359.000000,17.770000,1.000000,9.490000,1040.920000,15.340000,0.980000,7.000000,20.540000,0.990000


In [8]:
#Displaying the correlation between different variables.
# This helps to understand the relationship of all features with energy.Closer to 1 means the stronger the correlation.
# A minus sign indicates negative correlation between the two variables, which suggests that if one increases, the other lowers and vice versa.
ehw1.corr()

,energy_sum,temperatureMax,windBearing,dewPoint,cloudCover,windSpeed,pressure,visibility,humidity,uvIndex,temperatureMin,moonPhase
energy_sum,1.000000,-0.176828,-0.016783,-0.155790,0.069204,0.030870,-0.018380,-0.063464,0.082911,-0.145026,-0.169467,-0.005539
temperatureMax,-0.176828,1.000000,0.079388,0.864664,-0.399278,-0.163204,0.165724,0.309455,-0.430944,0.684600,0.902263,0.019055
windBearing,-0.016783,0.079388,1.000000,0.107149,-0.075132,0.026277,0.002145,0.245762,0.018723,-0.054711,0.080438,0.050949
dewPoint,-0.155790,0.864664,0.107149,1.000000,-0.098436,-0.111927,0.003021,0.058389,0.034748,0.463659,0.940549,0.009440
cloudCover,0.069204,-0.399278,-0.075132,-0.098436,1.000000,0.184628,-0.120980,-0.378816,0.484326,-0.292288,-0.132858,-0.078404
windSpeed,0.030870,-0.163204,0.026277,-0.111927,0.184628,1.000000,-0.283971,0.232969,-0.028218,-0.100875,-0.046914,0.021535
pressure,-0.018380,0.165724,0.002145,0.003021,-0.120980,-0.283971,1.000000,0.075146,-0.317830,0.137646,0.057517,0.047985
visibility,-0.063464,0.309455,0.245762,0.058389,-0.378816,0.232969,0.075146,1.000000,-0.615740,0.314559,0.206610,0.082556
humidity,0.082911,-0.430944,0.018723,0.034748,0.484326,-0.028218,-0.317830,-0.615740,1.000000,-0.576431,-0.224508,-0.021291
uvIndex,-0.145026,0.684600,-0.054711,0.463659,-0.292288,-0.100875,0.137646,0.314559,-0.576431,1.000000,0.588498,-0.008913


In [9]:
#Dtypes function shows the data types of each variable.
ehw1.dtypes

LCLid              object
time               object
energy_sum        float64
stdorToU           object
Acorn              object
Acorn_grouped      object
temperatureMax    float64
windBearing         int64
icon               object
dewPoint          float64
cloudCover        float64
windSpeed         float64
pressure          float64
visibility        float64
humidity          float64
uvIndex             int64
temperatureMin    float64
moonPhase         float64
dtype: object

In [10]:
#Changing data type from object to int.
ehw1.loc[:,'time']=  pd.to_datetime(ehw1['time']).dt.date
#Using label encoder to handel categorical values.
labelencoder = LabelEncoder()
ehw1['Acorn' +'_']= labelencoder.fit_transform(ehw1['Acorn'])
ehw1['Acorn_grouped' +'_']= labelencoder.fit_transform(ehw1['Acorn_grouped'])
ehw1['time' +'_']= labelencoder.fit_transform(ehw1['time'])
ehw1['stdorToU' +'_']= labelencoder.fit_transform(ehw1['stdorToU'])
ehw1['icon' +'_']= labelencoder.fit_transform(ehw1['icon'])
ehw1['LCLid' +'_']= labelencoder.fit_transform(ehw1['LCLid'])
features = ['Acorn', 'Acorn_grouped','time', 'stdorToU', 'icon','LCLid']  
ehw1 = ehw1.drop(features, axis = 1)

In [11]:
ehw1.dtypes

energy_sum        float64
temperatureMax    float64
windBearing         int64
dewPoint          float64
cloudCover        float64
windSpeed         float64
pressure          float64
visibility        float64
humidity          float64
uvIndex             int64
temperatureMin    float64
moonPhase         float64
Acorn_              int32
Acorn_grouped_      int32
time_               int32
stdorToU_           int32
icon_               int32
LCLid_              int32
dtype: object

In [12]:
# Getting X and Y from ehw1 with all features.
X1 = ehw1.iloc[:,1:18] 
Y1 = ehw1.iloc[:,0] 

In [13]:
#Splitting dataset in training and testing
X_train,X_test,y_train,y_test=train_test_split(X1,Y1,test_size=0.2,random_state=2005)

In [14]:
#Calculating Mean square error using Linear Regression without feature selection.
lin1 = LinearRegression()
lin1.fit(X_train,y_train)
lin1_y_pred = lin1.predict(X_test)
L1 = mean_squared_error(y_test,lin1_y_pred)
L1

77.46190105375814

In [15]:
#Calculating Mean square error using KNN Regression without feature selection.
knn1 = KNeighborsRegressor(n_neighbors=20)
knn1.fit(X_train, y_train)
knn1_y_pred = knn1.predict(X_test)
# Squared Mean Error score and Root Mean Squared Error:
K1 = mean_squared_error(y_test, knn1_y_pred)
K1

81.72467570243423

In [16]:
#Calculating Mean square error using Ridge Regression without feature selection.
ridg1 = Ridge(alpha=1.0)
ridg1.fit(X_train,y_train)
ridg1_y_pred = ridg1.predict(X_test)
R1 = mean_squared_error(y_test,ridg1_y_pred)
R1

77.4613168890915

## Using Feature Selection

In [17]:
#Duplicating dataset for selecting feautres 
ehw2=ehw1

In [18]:
#Getting X and Y from ehw2
X_FS= ehw2.iloc[:,1:18]  
Y_FS= ehw2.iloc[:,0] 
Y_FS=Y_FS.astype('int')

In [19]:
#Removing values that are negative because feature selection did not work with negative values.
X_FS = X_FS[(X_FS > 0).all(1)]

In [20]:
# Finding Index's of values that are non-negative in X_FS and getting their respective labels from Y_FS.
a = X_FS.index
b = []
for i in a:
    b+=[i]
Y_FS = Y_FS.loc[b]

In [21]:
X_FS

,temperatureMax,windBearing,dewPoint,cloudCover,windSpeed,pressure,visibility,humidity,uvIndex,temperatureMin,moonPhase,Acorn_,Acorn_grouped_,time_,stdorToU_,icon_,LCLid_
146199,29.86,215,15.92,0.23,2.17,1013.75,12.87,0.59,5,20.54,0.03,4,1,228,1,3,261
124898,9.42,308,3.48,0.55,1.02,1032.90,11.59,0.83,1,2.43,0.94,4,1,696,1,3,55
938752,16.53,226,10.86,0.34,5.29,998.60,12.42,0.80,1,9.99,0.65,4,1,657,1,5,298
121506,10.69,208,7.10,0.64,1.29,998.27,11.31,0.91,1,7.17,0.44,4,1,328,1,3,257
632366,16.22,357,9.39,0.79,4.21,1015.56,10.78,0.78,4,12.01,0.36,3,1,502,1,3,1409
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145301,17.24,196,10.02,0.51,3.13,1007.34,12.09,0.82,5,10.43,0.59,4,1,126,1,3,199
152216,16.96,172,10.87,0.61,3.00,1013.81,10.46,0.83,4,8.16,0.09,4,1,523,1,3,201
746957,4.93,41,2.55,0.86,4.34,1032.15,3.48,0.91,1,2.98,0.52,3,1,420,1,2,789
154785,11.30,27,5.77,0.60,2.35,1005.87,11.54,0.76,4,7.20,0.23,4,1,498,1,3,280


In [22]:
Y_FS

146199     7
124898     4
938752    38
121506     6
632366    16
          ..
145301    11
152216     6
746957    39
154785     4
153602     7
Name: energy_sum, Length: 4437, dtype: int32

In [23]:
# Selecting top features and calculating their score
top_features= SelectKBest(score_func=chi2, k=3)
top_fit= top_features.fit(X_FS,Y_FS)
Scores= pd.DataFrame(top_fit.scores_)
X_FS_Columns= pd.DataFrame(X_FS.columns)

In [24]:
# Sorting these scores with their column names
features_scores= pd.concat([X_FS_Columns, Scores], axis=1)
features_scores.columns= ['Features_name', 'Score']
features_scores.sort_values(by = 'Score')

,Features_name,Score
14,stdorToU_,0.000000
12,Acorn_grouped_,0.000000
7,humidity,1.775017
5,pressure,5.928223
3,cloudCover,6.960002
10,moonPhase,9.152149
15,icon_,15.175697
11,Acorn_,39.438546
4,windSpeed,53.528863
6,visibility,70.632340


In [25]:
#Getting X and Y with selected features.
X2 = X_FS.iloc[:,[16,13,1,0,9]]  
Y2 = Y_FS

In [26]:
X2

,LCLid_,time_,windBearing,temperatureMax,temperatureMin
146199,261,228,215,29.86,20.54
124898,55,696,308,9.42,2.43
938752,298,657,226,16.53,9.99
121506,257,328,208,10.69,7.17
632366,1409,502,357,16.22,12.01
...,...,...,...,...,...
145301,199,126,196,17.24,10.43
152216,201,523,172,16.96,8.16
746957,789,420,41,4.93,2.98
154785,280,498,27,11.30,7.20


In [27]:
Y2

146199     7
124898     4
938752    38
121506     6
632366    16
          ..
145301    11
152216     6
746957    39
154785     4
153602     7
Name: energy_sum, Length: 4437, dtype: int32

In [28]:
#Splitting dataset in training and testing
X_train,X_test,y_train,y_test=train_test_split(X2,Y2,test_size=0.2,random_state=2005)

In [29]:
#Calculating Mean square error using linear Regression with feature selection.
lin2 = LinearRegression()
lin2.fit(X_train,y_train)
lin2_y_pred = lin2.predict(X_test)
L2 = mean_squared_error(y_test,lin2_y_pred)
L2

49.96109415206586

In [30]:
#Calculating Mean square error using KNN Regression with feature selection.
knn2 = KNeighborsRegressor(n_neighbors=20)
knn2.fit(X_train,y_train)
knn2_y_pred = knn2.predict(X_test)
K2 = mean_squared_error(y_test,knn2_y_pred)
K2

49.67740427927928

In [31]:
#Calculating Mean square error using Ridge Regression with feature selection.
ridg2 = Ridge(alpha=1.0)
ridg2.fit(X_train,y_train)
ridg2_y_pred = ridg2.predict(X_test)
R2 = mean_squared_error(y_test,ridg2_y_pred)
R2

49.96108505720835

In [32]:
# Best Model among the models with and without feature selection.
Models = {"Linear Regression without feature selection ":L1,"Linear Regression with feature selection ":L2,
          "KNN Regression without feature selection ":K1,"KNN Regression with feature selection ":K2,
          "Ridge Regression without feature selection ":R1,"Ridge Regression with feature selection ":R2}
Best_Model_value = min(Models.values())
for i in Models :
    if Models[i]==min(Models.values()):
        print(i + 'is the Best Model for the dataset used.')

KNN Regression with feature selection is the Best Model for the dataset used.


In [33]:
#Predicted energy consumption using KNN regression:
print(knn2_y_pred)

[12.15  6.95  9.05 11.55  7.85 10.7   6.2  10.45  9.55 10.6  12.05  8.2
  8.9  14.45  7.9   8.4   8.55  7.05 13.15 11.5   9.9  14.7   6.85  8.35
  7.35  6.7   8.65  9.25  8.75  8.4   6.05  6.95 16.3   8.   11.25 13.15
 19.45  8.35  8.35  9.05  5.   10.95  8.4   5.8   7.4   7.5   6.15  7.9
 12.65  9.2   6.8   7.45 17.5   8.6   6.45 13.    7.8   8.05  7.25  6.95
  7.2   5.25  7.9  13.4   7.75  6.8   7.   11.4  18.    5.45  8.95 21.1
  9.7   7.35 12.15  7.2  10.9  12.55  9.2   7.95 14.85  5.3   7.9   9.7
 10.75  7.4  10.15 11.75 14.7   8.55 10.55  8.5  10.2  11.25  5.45  8.5
  8.6  11.35  6.2   6.7   9.65 11.15  8.7   8.2  10.55 10.1   6.35 12.7
  6.45  7.25 16.35  6.6  10.1   9.75  8.9  12.9   7.55  8.9   8.2   8.25
  8.4   7.35  7.75  8.8  13.45  6.    8.8   6.    9.55 10.75 12.2   9.65
  6.9  10.6   7.25 10.45 11.4   7.6  13.25  6.8   7.7   7.7   9.15 19.25
 17.25  7.25 10.1   8.9  15.05 10.55 11.5   9.4  12.5   7.8  11.2   8.8
  9.4   7.8   7.45  8.95  6.35 10.2  15.05  9.4   8.1   6.

## Reference :


https://365datascience.com/tutorials/python-tutorials/predictive-model-python/

https://scikit-learn.org/stable/modules/feature_selection.html

https://github.com/Gaelim/PowerBI_Python_A_to_Z